In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import scipy as sp
from statsmodels.stats.multitest import multipletests
import os
from os.path import join
import time

from anndata import read_h5ad

# scTRS tools
import scdrs

# autoreload
%load_ext autoreload
%autoreload 2

### Check MAGMA file

In [62]:
df_magma = pd.read_csv('/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/gene_annotation/MAGMA-v108/'
                       'MAGMA_v108_GENE_10_ZSTAT_for_scDRS.txt', sep='\t', index_col=0)
df_gene = pd.read_csv('/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/gene_annotation/MAGMA_file/'
                       'NCBI37.3.gene.loc', sep='\t', header=None)
dic_num2gene = {x:y for x,y in zip(df_gene[0], df_gene[5])}
df_trait = pd.read_csv('/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/supp_table.rv1/trait_info.tsv', 
                       sep='\t')

MAGMA_FILE = '/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/gene_annotation/GENE_CELL_10/'\
    '@t.sumstats.gene_trait.@s'
for trait in df_trait['Trait_Identifier']:
    gene_out = MAGMA_FILE.replace('@t', trait).replace('@s', 'genes.out')
    gene_raw = MAGMA_FILE.replace('@t', trait).replace('@s', 'genes.raw')
    if os.path.exists(gene_out) is False:
        print('Missing', gene_out)
        continue
    if os.path.exists(gene_out) is False:
        print('Missing', gene_raw)
        continue
    
    # Check consistency
    df_gene_out = pd.read_csv(gene_out, delim_whitespace=True)
    df_gene_out['SYM'] = [dic_num2gene[x] for x in df_gene_out['GENE']]
    df_gene_out.index = df_gene_out['SYM']
    
    max_abs_dif = np.absolute(df_magma.loc[df_gene_out['SYM'], trait] - df_gene_out['ZSTAT']).max()
    print(trait, max_abs_dif)
#     break

PASS_CD_deLange2017 0.0
PASS_Celiac 0.0
PASS_IBD_deLange2017 0.0
PASS_Lupus 0.0
PASS_Multiple_sclerosis 0.0
PASS_Primary_biliary_cirrhosis 0.0
PASS_Rheumatoid_Arthritis 0.0
PASS_UC_deLange2017 0.0
UKB_460K.blood_EOSINOPHIL_COUNT 0.0
UKB_460K.blood_LYMPHOCYTE_COUNT 0.0
UKB_460K.blood_MEAN_CORPUSCULAR_HEMOGLOBIN 0.0
UKB_460K.blood_MONOCYTE_COUNT 0.0
UKB_460K.blood_PLATELET_COUNT 0.0
UKB_460K.blood_RBC_DISTRIB_WIDTH 0.0
UKB_460K.blood_RED_COUNT 0.0
UKB_460K.blood_WHITE_COUNT 0.0
UKB_460K.disease_AID_ALL 0.0
UKB_460K.disease_ALLERGY_ECZEMA_DIAGNOSED 0.0
UKB_460K.disease_ASTHMA_DIAGNOSED 0.0
UKB_460K.disease_HYPOTHYROIDISM_SELF_REP 0.0
UKB_460K.disease_RESPIRATORY_ENT 0.0
PASS_ADHD_Demontis2018 0.0
PASS_Alzheimers_Jansen2019 0.0
PASS_BIP_Mullins2021 0.0
PASS_DrinksPerWeek_Liu2019 0.0
PASS_GeneralRiskTolerance_KarlssonLinner2019 0.0
PASS_Insomnia_Jansen2019 0.0
PASS_Intelligence_SavageJansen2018 0.0
PASS_MDD_Howard2019 0.0
PASS_ReactionTime_Davies2018 0.0
PASS_SWB 0.0
PASS_Schizophrenia_Pard

### Make GS file 

In [2]:
# Check consistency
GS_FILE='/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/gs_file/magma_10kb_top1000_zscore.all_traits.rv1.gs'
GS_FILE_REF='/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/gs_file/magma_10kb_1000.74_traits.gs'

dic_gs = scdrs.util.load_gs(GS_FILE)
dic_gs_ref = scdrs.util.load_gs(GS_FILE_REF)

trait_list = list(set(dic_gs) & set(dic_gs_ref))
print('n_trait_overlap=%d'%len(trait_list))

print('|{:^50s}|{:^10s}|'.format('TRAIT', 'Overlap'))
for trait in trait_list:
    print('|{:^50s}|{:^10d}|'.format(trait, len(set(dic_gs[trait][0]) & set(dic_gs_ref[trait][0]))))

n_trait_overlap=74
|                      TRAIT                       | Overlap  |
|              UKB_460K.cov_EDU_YEARS              |   1000   |
|               PASS_UC_deLange2017                |   1000   |
|              UKB_460K.pigment_HAIR               |   1000   |
|              UKB_460K.cancer_BREAST              |   1000   |
|           UKB_460K.biochemistry_HbA1c            |   1000   |
|   PASS_GeneralRiskTolerance_KarlssonLinner2019   |   1000   |
|           PASS_ReactionTime_Davies2018           |   999    |
|          PASS_Primary_biliary_cirrhosis          |   1000   |
|                UKB_460K.body_BMIz                |   1000   |
|             UKB_460K.disease_AID_ALL             |   1000   |
|           PASS_Coronary_Artery_Disease           |   1000   |
|                    PASS_Lupus                    |   1000   |
|     UKB_460K.biochemistry_Testosterone_Male      |   1000   |
|        PASS_Intelligence_SavageJansen2018        |   1000   |
|            UKB_460K

In [5]:
# Check correctness for one trait
trait = 'UKB_460K.bmd_HEEL_TSCOREz'
df_magma = pd.read_csv('/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data'
                       '/gene_annotation/MAGMA-v108/MAGMA_v108_GENE_10_ZSTAT_for_scDRS.txt', sep='\t')
df_magma = df_magma[[trait]].dropna().sort_values(trait, ascending=False)

# Top 100,500,1000,2000
for weight in ['zscore', 'uniform']:
    for ngene in [100,500,1000,2000]:
        gs_name = 'magma_10kb_top%d_%s.all_traits.rv1.gs'%(ngene,weight)
        dic_gs = scdrs.util.load_gs('/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/gs_file/%s'%gs_name)
        n_gene_gs = len(dic_gs[trait][0])
        v_w = df_magma[trait][:n_gene_gs] if weight=='zscore' else  np.ones(n_gene_gs)
        print('{:^60s} {:^20s} {:^30s}'.format(gs_name, 
                'overlap=%d/%d'%(len(set(df_magma.index[:ngene]) & set(dic_gs[trait][0])), 
                                 len(dic_gs[trait][0])
                                ),
                'weight_dif=%0.2e'%np.mean(np.absolute( v_w - np.array(dic_gs[trait][1]) ))))
        
    # FDR<0.01
    ngene = multipletests(scdrs.util.zsc2pval(df_magma[trait].values), alpha=0.01, method="fdr_bh")[0].sum()
    gs_name = 'magma_10kb_fdr001_cap100n2000_%s.all_traits.rv1.gs'%(weight)
    dic_gs = scdrs.util.load_gs('/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/gs_file/%s'%gs_name)
    n_gene_gs = len(dic_gs[trait][0])
    v_w = df_magma[trait][:n_gene_gs] if weight=='zscore' else  np.ones(n_gene_gs)
    print('{:^60s} {:^20s} {:^30s}'.format(gs_name, 
            'overlap=%d/%d'%(len(set(df_magma.index[:ngene]) & set(dic_gs[trait][0])), 
                             len(dic_gs[trait][0])
                            ),
            'weight_dif=%0.2e'%np.mean(np.absolute( v_w - np.array(dic_gs[trait][1]) ))))
    
    # FWER<0.05
    ngene = multipletests(scdrs.util.zsc2pval(df_magma[trait].values), alpha=0.05, method="bonferroni")[0].sum()
    gs_name = 'magma_10kb_fwer005_cap100n2000_%s.all_traits.rv1.gs'%(weight)
    dic_gs = scdrs.util.load_gs('/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/gs_file/%s'%gs_name)
    n_gene_gs = len(dic_gs[trait][0])
    v_w = df_magma[trait][:n_gene_gs] if weight=='zscore' else  np.ones(n_gene_gs)
    print('{:^60s} {:^20s} {:^30s}'.format(gs_name, 
            'overlap=%d/%d'%(len(set(df_magma.index[:ngene]) & set(dic_gs[trait][0])), 
                             len(dic_gs[trait][0])
                            ),
            'weight_dif=%0.2e'%np.mean(np.absolute( v_w - np.array(dic_gs[trait][1]) ))))

         magma_10kb_top100_zscore.all_traits.rv1.gs             overlap=99/100         weight_dif=2.58e-03      
         magma_10kb_top500_zscore.all_traits.rv1.gs            overlap=500/500         weight_dif=5.16e-04      
        magma_10kb_top1000_zscore.all_traits.rv1.gs            overlap=973/1000        weight_dif=2.58e-04      
        magma_10kb_top2000_zscore.all_traits.rv1.gs           overlap=2000/2000        weight_dif=1.29e-04      
   magma_10kb_fdr001_cap100n2000_zscore.all_traits.rv1.gs     overlap=2000/2000        weight_dif=1.29e-04      
  magma_10kb_fwer005_cap100n2000_zscore.all_traits.rv1.gs     overlap=2000/2000        weight_dif=1.29e-04      
        magma_10kb_top100_uniform.all_traits.rv1.gs             overlap=99/100         weight_dif=0.00e+00      
        magma_10kb_top500_uniform.all_traits.rv1.gs            overlap=500/500         weight_dif=0.00e+00      
        magma_10kb_top1000_uniform.all_traits.rv1.gs           overlap=973/1000        weight_di

In [ ]:
# Create batch files
df_trait = pd.read_csv('/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/supp_table/trait_info.tsv', sep='\t')
trait_list = list(df_trait['Trait_Identifier']) + ['PASS_BIP_Mullins2021']
print('n_trait=%d'%len(trait_list))

BATCH_SIZE=5

# zscore
GS_FILE='/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/gs_file/magma_10kb_top1000_zscore.all_traits.rv1.gs'
OUT_FOLDER='/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/gs_file/magma_10kb_top1000_zscore.75_traits.batch'
df_gs = pd.read_csv(GS_FILE, sep='\t')
df_gs = df_gs.loc[[x in trait_list for x in df_gs['TRAIT']]]
for i_batch in range(np.ceil(df_gs.shape[0]/BATCH_SIZE).astype(int)):
    df_gs.iloc[i_batch*BATCH_SIZE:(i_batch+1)*BATCH_SIZE].to_csv(
        OUT_FOLDER+'/batch%d.gs'%i_batch, sep='\t', index=False)
    
# uniform
GS_FILE='/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/gs_file/magma_10kb_top1000_uniform.all_traits.rv1.gs'
OUT_FOLDER='/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/gs_file/magma_10kb_top1000_uniform.75_traits.batch'
df_gs = pd.read_csv(GS_FILE, sep='\t')
df_gs = df_gs.loc[[x in trait_list for x in df_gs['TRAIT']]]
for i_batch in range(np.ceil(df_gs.shape[0]/BATCH_SIZE).astype(int)):
    df_gs.iloc[i_batch*BATCH_SIZE:(i_batch+1)*BATCH_SIZE].to_csv(
        OUT_FOLDER+'/batch%d.gs'%i_batch, sep='\t', index=False)

In [ ]:
# Get .gs file for the updated 74 traits
df_gs = pd.read_csv('/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/gs_file/'
                    'magma_10kb_top1000_zscore.all_traits.rv1.gs', sep='\t')
df_trait = pd.read_csv('/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/supp_table.rv1/trait_info.tsv',
                       sep='\t')

df_gs_small = df_gs.loc[[x in df_trait['Trait_Identifier'].values for x in df_gs['TRAIT']]].copy()
df_gs_small.sort_values('TRAIT', inplace=True)
df_gs_small.to_csv('/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/gs_file/'
                   'magma_10kb_top1000_zscore.74_traits.rv1.gs', sep='\t', index=False)

### .gs file for pairwise exclusion gene sets

In [2]:
DIC_TRAIT_LIST = {}
DIC_TRAIT_LIST['immune'] = [
    'PASS_IBD_deLange2017', 
    'PASS_CD_deLange2017',
    'PASS_UC_deLange2017', 
    'PASS_Rheumatoid_Arthritis', 
    'PASS_Multiple_sclerosis', 
    'UKB_460K.disease_AID_ALL', 
    'UKB_460K.disease_HYPOTHYROIDISM_SELF_REP',
    'UKB_460K.disease_ALLERGY_ECZEMA_DIAGNOSED',
    'UKB_460K.disease_ASTHMA_DIAGNOSED', 
    'UKB_460K.disease_RESPIRATORY_ENT'
]
DIC_TRAIT_LIST['brain'] = [
    'PASS_BIP_Mullins2021', 
    'PASS_MDD_Howard2019', 
    'PASS_Schizophrenia_Pardinas2018', 
    'UKB_460K.mental_NEUROTICISM',
    'UKB_460K.cov_EDU_COLLEGE',
    'UKB_460K.body_BMIz',
    'UKB_460K.cov_SMOKING_STATUS'    
]
DIC_TRAIT_LIST['metabolic'] = [
    'UKB_460K.biochemistry_Triglycerides', 
    'UKB_460K.biochemistry_HDLcholesterol',
    'UKB_460K.biochemistry_LDLdirect',
    'UKB_460K.biochemistry_Cholesterol', 
    'UKB_460K.biochemistry_Testosterone_Male', 
    'UKB_460K.biochemistry_AlanineAminotransferase',
    'UKB_460K.biochemistry_AlkalinePhosphatase',
    'UKB_460K.biochemistry_SHBG', 
    'UKB_460K.biochemistry_TotalBilirubin'
]
trait_list = DIC_TRAIT_LIST['immune'] + DIC_TRAIT_LIST['brain'] + DIC_TRAIT_LIST['metabolic']

df_gs = pd.read_csv('/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/gs_file/'
                    'magma_10kb_top1000_zscore.74_traits.rv1.gs', sep='\t', index_col=0)

df_gs_pw_exc = pd.DataFrame(columns=['TRAIT', 'GENESET'])
i_trait = 0
for trait1 in trait_list:
    for trait2 in trait_list:
        if trait1==trait2:
            continue
        dic_weight = {x.split(':')[0]:x.split(':')[1] for x in df_gs.loc[trait1, 'GENESET'].split(',')}
        gene_list_exc = set([x.split(':')[0] for x in df_gs.loc[trait2, 'GENESET'].split(',')])

        gene_list = sorted(set(dic_weight) - set(gene_list_exc))
        gene_weights = [dic_weight[g] for g in gene_list]
        geneset_str = ",".join([f"{g}:{w}" for g, w in zip(gene_list, gene_weights)])
        df_gs_pw_exc.loc[i_trait] = ['%s_exc_%s'%(trait1,trait2), geneset_str]
        i_trait += 1
            
df_gs_pw_exc.to_csv('/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/gs_file/'
                   'magma_10kb_top1000_zscore.domain_traits_pairwise_exclusion.rv1.gs', sep='\t', index=False)

In [14]:
# Create batch file
BATCH_SIZE=5

# zscore
GS_FILE='/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/gs_file/magma_10kb_top1000_zscore.domain_traits_pairwise_exclusion.rv1.gs'
OUT_FOLDER='/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/gs_file/magma_10kb_top1000_zscore.domain_traits_pairwise_exclusion.batch'
df_gs = pd.read_csv(GS_FILE, sep='\t')

trait_list_exc = [x.replace('.score.gz', '') for x in os.listdir(
    '/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/score_file/'
    'score.tms_facs_with_cov.magma_10kb_top1000_zscore_pairwise_exclusion')]
df_gs = df_gs.loc[~df_gs['TRAIT'].isin(trait_list_exc)]

for i_batch in range(np.ceil(df_gs.shape[0]/BATCH_SIZE).astype(int)):
    df_gs.iloc[i_batch*BATCH_SIZE:(i_batch+1)*BATCH_SIZE].to_csv(
        OUT_FOLDER+'/batch%d.gs'%i_batch, sep='\t', index=False)
    

### Sumsampled gene sets

In [16]:
# temp_df = pd.read_csv(
#     '/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/gene_annotation/MAGMA-v108/'
#     'MAGMA_v108_GENE_10_PSTAT_for_scDRS_subsampled.txt', sep='\t')
# temp_df.columns = [x.replace('.sumstats','') for x in temp_df]
# temp_df.index.name = 'GENE'
# temp_df.to_csv('/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/gene_annotation/MAGMA-v108/'
#                'MAGMA_v108_GENE_10_PSTAT_for_scDRS_subsampled_refmt.txt', sep='\t')

# temp_df = pd.read_csv(
#     '/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/gene_annotation/MAGMA-v108/'
#     'MAGMA_v108_GENE_10_ZSTAT_for_scDRS_subsampled.txt', sep='\t')
# temp_df.columns = [x.replace('.sumstats','') for x in temp_df]
# temp_df.index.name = 'GENE'
# temp_df.to_csv('/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/gene_annotation/MAGMA-v108/'
#                'MAGMA_v108_GENE_10_ZSTAT_for_scDRS_subsampled_refmt.txt', sep='\t')

In [22]:
# Create batch file
BATCH_SIZE=5

# zscore
GS_FILE='/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/gs_file/magma_10kb_top1000_zscore.UKB_subsample.rv1.gs'
OUT_FOLDER='/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/gs_file/magma_10kb_top1000_zscore.UKB_subsample.rv1.batch'
df_gs = pd.read_csv(GS_FILE, sep='\t')

for i_batch in range(np.ceil(df_gs.shape[0]/BATCH_SIZE).astype(int)):
    df_gs.iloc[i_batch*BATCH_SIZE:(i_batch+1)*BATCH_SIZE].to_csv(
        OUT_FOLDER+'/batch%d.gs'%i_batch, sep='\t', index=False)
    

### UKB145K gene sets

In [2]:
# Create batch file
BATCH_SIZE=5

# zscore
GS_FILE='/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/gs_file/magma_10kb_top1000_zscore.UKB145K_26trait.rv1.gs'
OUT_FOLDER='/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/gs_file/magma_10kb_top1000_zscore.UKB145K_26trait.rv1.batch'
df_gs = pd.read_csv(GS_FILE, sep='\t')

for i_batch in range(np.ceil(df_gs.shape[0]/BATCH_SIZE).astype(int)):
    df_gs.iloc[i_batch*BATCH_SIZE:(i_batch+1)*BATCH_SIZE].to_csv(
        OUT_FOLDER+'/batch%d.gs'%i_batch, sep='\t', index=False)
    

### Cell cycle genes 

In [8]:
cc_genes = pd.read_csv('/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/gene_annotation/'
                       'Macosko_cell_cycle_genes.txt', sep='\t')
df_gs = pd.DataFrame(columns=['TRAIT', 'GENESET'])
df_gs.loc[1] = ['cell_cycle_S', ','.join(cc_genes['S'].dropna())]
df_gs.loc[2] = ['cell_cycle_G2M', ','.join(cc_genes['G2.M'].dropna())]
df_gs.to_csv('/n/holystore01/LABS/price_lab/Users/mjzhang/scDRS_data/gs_file/cell_cycle.gs',
             sep='\t', index=False)

In [7]:
df_gs

,TRAIT,GENESET
1,cell_cycle_S,"ABCC5,ABHD10,ANKRD18A,ASF1B,ATAD2,BBS2,BIVM,BL..."
2,cell_cycle_G2M,"ANLN,AP3D1,ARHGAP19,ARL4A,ARMC1,ASXL1,ATL2,AUR..."


In [3]:
cc_genes

,IG1.S,S,G2.M,M,M.G1,Unnamed: 5
0,ACD,ABCC5,ANLN,AHI1,AGFG1,NaN
1,ACYP1,ABHD10,AP3D1,AKIRIN2,AGPAT3,NaN
2,ADAMTS1,ANKRD18A,ARHGAP19,ANKRD40,AKAP13,NaN
3,ANKRD10,ASF1B,ARL4A,ANLN,AMD1,NaN
4,APEX2,ATAD2,ARMC1,ANP32B,ANP32E,NaN
...,...,...,...,...,...,...
146,NaN,NaN,NaN,YWHAH,NaN,NaN
147,NaN,NaN,NaN,ZC3HC1,NaN,NaN
148,NaN,NaN,NaN,ZFX,NaN,NaN
149,NaN,NaN,NaN,ZMYM1,NaN,NaN


### Old code